In [1]:
import torch
import sophius.utils as utils
import sophius.dataloader as dload
import torchvision.datasets as dset
import torchvision.transforms as T

%load_ext autoreload
%autoreload 2

NUM_VAL = 1024

cifar10 = dset.CIFAR10('data//CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [43]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, NUM_VAL)

In [29]:
from sophius.modelgen import ConvFCModelGenerator, ConvGAPModelGenerator
import torch

model_gen = ConvFCModelGenerator((3, 32, 32), 10, conv_num = 6, lin_num = 2)
model_tmpl = model_gen.generate_model_tmpl()
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(model_tmpl)

Conv2d       (192, 8, 8)    (5, 5)   (4, 4)  
AvgPool2d    (192, 5, 5)    (4, 4)   (1, 1)  
LeakyReLU    (192, 5, 5)    (0.001) 
Conv2d       (64, 2, 2)     (5, 5)   (4, 4)  
ReLU         (64, 2, 2)    
Conv2d       (192, 1, 1)    (4, 4)   (3, 3)  
MaxPool2d    (192, 1, 1)    (2, 2)   (2, 2)  
AvgPool2d    (192, 1, 1)    (2, 2)   (2, 2)  
PReLU        (192, 1, 1)   
AvgPool2d    (192, 2, 2)    (4, 4)   (1, 1)  
Flatten      768           
Linear       10            



In [66]:
model_gen = ConvGAPModelGenerator((3, 32, 32), 10, conv_num = 6)
model_tmpl = model_gen.generate_model_tmpl()
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(model_tmpl)

Conv2d       (96, 32, 32)   (3, 3)   (1, 1)  
PReLU        (96, 32, 32)  
BatchNorm2d  (96, 32, 32)  
Conv2d       (96, 11, 11)   (4, 4)   (3, 3)  
LeakyReLU    (96, 11, 11)   (0.01)  
AvgPool2d    (96, 11, 11)   (3, 3)   (1, 1)  
MaxPool2d    (96, 3, 3)     (4, 4)   (3, 3)  
Conv2d       (64, 2, 2)     (2, 2)   (3, 3)  
MaxPool2d    (64, 1, 1)     (2, 2)   (1, 1)  
AvgPool2d    (64, 2, 2)     (4, 4)   (1, 1)  
PReLU        (64, 2, 2)    
ReLU         (64, 2, 2)    
GlobalAvgPool2d (64, 1, 1)    
Flatten      64            
Linear       10            



In [77]:
from sophius.train import train_express_gpu
loader_gpu = dload.get_loader_gpu(cifar_gpu, NUM_VAL)
# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 3.8s 
val_acc: 0.343, train_acc: 0.322
